In [1]:
!pip install pymysql # mysql database connection library

In [2]:
!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

Traceback (most recent call last):
  File "/usr/local/bin/jupyter-notebook", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/jupyter_core/application.py", line 264, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/traitlets/config/application.py", line 845, in launch_instance
    app.initialize(argv)
  File "/usr/local/lib/python3.7/dist-packages/traitlets/config/application.py", line 88, in inner
    return method(app, *args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/notebook/notebookapp.py", line 1507, in initialize
    self.init_webapp()
  File "/usr/local/lib/python3.7/dist-packages/notebook/notebookapp.py", line 1297, in init_webapp
    self.http_server.listen(port, self.ip)
  File "/usr/local/lib/python3.7/dist-packages/tornado/tcpserver.py", line 143, in listen
    sockets = bind_sockets(port, address=address)
  File "/usr/local/lib/pytho

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
import pymysql.cursors  # For Request Query String
import pandas as pd # Querying results to make dataframe(to divide datas)
import time

conn = pymysql.connect(host='csrd.cksngv0eixsu.ap-northeast-2.rds.amazonaws.com', port=3306, user='admin', password='qwerty1!', \
db='before-pp', charset='utf8', autocommit=True, cursorclass=pymysql.cursors.DictCursor)        # mysql connect inform

In [5]:
# import sys

# sys.argv[0] = '"'
# sql = "SELECT * FROM bf_pl a join bf_tg b on a.pl_id = b.pl_id join bf_sg c on c.pl_id = b.pl_id WHERE pl_intro LIKE :name group by 1",  {"name": "%클래식%"}
#     # select data in DB 
# print(sql)

# # cur = conn.cursor() # open to query
# # cur.execute(sql)    # excute query
# # result = cur.fetchall() # merging result
# # conn.close()    # DB disconnect
# # print(result)   # Check Status
# # # An error appeared, but it doesn't matter because just too many search results.

In [ ]:
cur = conn.cursor() # open to query
sql = "SELECT a.pl_name, a.pl_intro,  b.tg_name, c.sg_name, c.sg_artists FROM bf_pl a join bf_tg b on a.pl_id = b.pl_id join bf_sg c on c.pl_id = b.pl_id;"  # select data in DB
cur.execute(sql)    # excute query

In [ ]:
result = cur.fetchall() # merging result
conn.close()    # DB disconnect
print(result)   # Check Status
# An error appeared, but it doesn't matter because just too many search results.

In [ ]:
df = pd.DataFrame(result)   # 전체 음악리스트 확인(playlist_id 기준이기 때문에 중복값 들어있음)
df

In [ ]:
df.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=True)  # 중복 처리
df

In [ ]:
df['컬럼명'].str.contains('검색하고자 하는 문자열')

In [ ]:
df1 = df[1800:2200]  # 너무 많은 양의 데이터를 처리하면 Youtube에서 IP ban을 하기 때문과 테스트를 위해 슬라이싱 
df1.sort_values(by=['sg_name'], axis=0)

In [ ]:
df1.info()

In [ ]:
ext_lists = df1.values.tolist() # pytube에서 검색어로 사용하기 위한 리스트 변환

In [ ]:
!python3 -m pip install --upgrade "git+https://github.com/nficano/pytube.git"   # pytube-12.0.0 설치

In [ ]:
import os
import pytube
from pytube import Search


audio_lists = []    # 검색결과가 나온 비디오 넣는 리스트
forbidden_lists = []    # 검색결과 없는 비디오 리스트 - 추후 DB 적용하기 위하여

for ext in ext_lists:
    try:
        StrClean = "-".join(ext)    # 노래 제목과 가수를 "-"를 구분자로 문자열 합치기
        yt = Search(StrClean).results[0]    # Search 메소드를 사용하여 검색하고, 그 결과의 1번째 결과 사용(빈도 상, 1 ~ 2번째 영상에 가장 관련있는 영상 표시)
    except:
        print("=================== Error Occured :  " + StrClean + " ===================")  # 에러 로그
        forbidden_lists.append(StrClean)    # 검색결과 없는 비디오 리스트 삽입
    audio_lists.append({StrClean : yt})     # 성공적으로 검색되는 비디오의 Youtube 객체 리스트 삽입
    # 추후 파일명으로 사용하여 라벨링 및 DB 적용을 수월하게 하기 위하여 딕셔너리 형태로 전달

In [ ]:
audio_lists, len(audio_lists)   # 성공 목록 확인 및 개수 확인

In [ ]:
forbidden_lists, len(forbidden_lists)   # 실패 목록 확인 및 개수 확인

In [ ]:
# import csv
 
# f = open("data.csv", "w")
# writer = csv.writer(f)
 
# writer.writerows(audio_lists) ## 여기 주목!
# f.close()

In [ ]:
# import csv
# fieldnames = ['url']
# f = open("data.csv", "w")
# writer = csv.DictWriter(f, fieldnames=fieldnames)
# # writer.writeheader()
# writer.writerows(audio_lists)

# for row in audio_lists:
#     writer.writerow(row)
# f.close()

In [ ]:
# import csv
 
# f = open("data.csv", "r")
# reader = csv.reader(f)
 
# audio_lists2 = list(reader)

In [ ]:
keys = list(audio_lists[0].items())[0][0]   #   딕셔너리 언패킹을 위한 자료형 확인용
link = audio_lists[0].get(keys)
print(f"{keys}의 자료형은 '{type(keys)}'// {link}의 자료형은 '{type(link)}'")

In [ ]:
import time     # 딜레이 주기 위함
unfounded_lists = []    # 검색된 영상 중 필터링된, 관련성 떨어지는 목록용 리스트

def youtube_download(var):
    keys = str(list(var[0].items())[0][0])   # 딕셔너리에서 파일 이름으로 쓸 string 언패킹
    link = var.get(keys)                 # 딕셔너리에서 파일이름이 key값이므로, values 값인  (pytube.__main__.YouTube) 자료형 언패킹
    audio_streams = link.streams.filter(only_audio=True)    # pytube에서 지원하는 다양한 stream 중 Audio 속성을 가진 stream만
    # print(audio_streams)  # audio streams 처리 전체 정보

    flt1a = (int(link.length)) > 130    # 영상 길이 2분 10초 초과
    flt1b = (int(link.length)) < 600    # 영상 길이 10분 미만
    # flt1b = (int(link.length)) < 150    # 일부러 에러를 내기 위한 조건

    flt1 = flt1a and flt1b              # 위의 두 조건 동시에 만족하는 필터
    flt2 = ("official" in str(link.title).lower() or "audio" in str(link.title).lower() or "lylic" in str(link.title).lower())  
    # "official", "audio", "lylic" 중 제목에 하나라도 들어가는 영상 제목 통과

    # 내용 체크
    print(f'영상제목: {link.title},  영상 조회수: {link.views},  영상 길이: {link.length} sec. [{str(link.length // 60).zfill(2)}:{str(link.length % 60).zfill(2)}]')
    print(f'영상 URL: {link.watch_url}')
    print(f"# 1번 체크포인트 - 음원 길이(초):  {int(link.length)}, 타입은? : {type(int(link.length))}")
    print(f"# 2번 체크포인트 - 음원 제목(소문자):  {str(link.title).lower()}") 
    print(f"# 3번 체크포인트 - 2분 이상 10분 이하 길이:  {flt1}  ,  음원에 자주 붙은 문자열 포함 여부: {flt2}")

    # 저장 폴더 만들기 / Local 혹은 Server 환경에서 돌릴 경우 사용
    # if not os.path.exists('aurigin'):
    #     os.mkdir('aurigin')
    # else:
    #     pass

    ## filter()를 활용하여 파일 확장자 타입 등을 포함한 streams 처리
    if flt1 or flt2 == True:    # 위의 1, 2 조건 중 한가지라도 만족한다면 다운로드
        audio_streams = link.streams.filter(file_extension='mp4').get_by_itag(140)  # 여러개의 다운로드 스트림 선택해서 링크 하나 가져오기
        # audio_streams - 예시: <Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
        # print(audio_streams) # 가능한 stream 표시        

        title = keys  # 영상 제목으로 파일명이 하고 싶다면? audio_streams.title 또는 link.title 사용

        # special_char = '\/:*?"<>|.'
        # for c in special_char:
        #     if c in title:
        #         title = title.replace(c, '')
        # print(title)  # 파일명에 혹시라도 특수문자 들어갈까봐 Regex마냥 사용할 수 있는거.. 왜 근데 이사람은 정규식 안 썼지?
    
        audio_streams.download(filename= title + '.mp3', output_path='/content/drive/Othercomputers/내 노트북/Project/Dataset/songs/download')     # 설정한 파일명, 경로에 다운로드
        time.sleep(5)                                                          # 5초 휴지
        # print(f"=============== {keys.spilt('-')[1]}란 가수의 {keys.spilt('-')[0]} 란 노래 음원 다운로드 완료! ===============")    # 성공 로그
        print(f"#@#@#@#@#@#@#@ {keys} 다운로드 성공!")             # 성공 로그
        print("============================================== 구분선 ==============================================")
    else:
        print(f"^^^^^^^^^^^ Related Video Not Found : {keys}")  # 실패 로그
        unfounded_lists.append(keys)    # 실패 목록 리스트에 저장
        print("============================================== 구분선 ==============================================")
        

In [ ]:
fail_cnt = 0
for fail_cnt ,audio in enumerate(audio_lists):
    try:
        youtube_download(audio)
    except:
        print(f"no error keep going, {fail_cnt}개째 다운로드 실패")

In [ ]:
def unfounded_df(ufs):   # unfounded_lists를 DB에 적용하기 위한 Dataframe 만들기
    df = pd.DataFrame(ufs, columns = ['col'])
    df['sg_name'] = df.col.str.split('-').str[0]
    df['sg_artists'] = df.col.str.split('-').str[1]
    df.drop(['col'], axis=1, inplace=True)
    return df

In [ ]:
failed_df = unfounded_df(unfounded_lists)   # 데이터프레임 확인
failed_df

In [ ]:
ext_lists2 = failed_df.values.tolist() # pytube에서 검색어로 사용하기 위한 리스트 변환

In [ ]:
import os
import pytube
from pytube import Search


audio_lists2 = []    # 검색결과가 나온 비디오 넣는 리스트
forbidden_lists2 = []    # 검색결과 없는 비디오 리스트 - 추후 DB 적용하기 위하여

for ext in ext_lists2:
    try:
        StrClean = "-".join(ext)    # 노래 제목과 가수를 "-"를 구분자로 문자열 합치기
        yt = Search(StrClean).results[1]    # Search 메소드를 사용하여 검색하고, 1번째 영상에서 에러가 발생했으므로 2번째 영상 선택
    except:
        print("=================== Error Occured :  " + StrClean + " ===================")  # 에러 로그
        forbidden_lists2.append(StrClean)    # 검색결과 없는 비디오 리스트 삽입
    audio_lists2.append({StrClean : yt})     # 성공적으로 검색되는 비디오의 Youtube 객체 리스트 삽입
    # 추후 파일명으로 사용하여 라벨링 및 DB 적용을 수월하게 하기 위하여 딕셔너리 형태로 전달

In [ ]:
forbidden_lists2

In [ ]:
fail_cnt = 0
for fail_cnt ,audio in enumerate(audio_lists2):
    try:
        youtube_download(audio)
    except:
        print(f"no error keep going, {fail_cnt}개째 다운로드 실패")

In [ ]:
# !zip -r /content/test200_songs.zip /content/test        # 다운로드 받은 음악 로컬로 저장하기 위해 압축

In [ ]:
# !ls -alh 

In [ ]:
# from google.colab import files
# files.download('test200_songs.zip')

In [ ]:
#  !rm -rf *

In [ ]:
# !ls

In [ ]:
# test = list()
# def slicing_loop(df, col):
#     for i in df.index:
#         val = df.loc[i : (i + 100)]
#         test.append(val)

# slicing_loop(df1, col)
# test[::100]